In [1]:
%cd ..

/home/nikita/edu/competitions/admet


In [2]:
import pandas as pd
import numpy as np
from rdkit import Chem, RDLogger
from rdkit.Chem import Descriptors
from tqdm import tqdm
import warnings
from catboost import CatBoostClassifier

df = pd.read_csv("data/train_admet.csv")

In [3]:
df = df.drop("Unnamed: 0", axis=1)
df

,Drug_ID,Drug,Y,property
0,0,O=[N+]([O-])c1c2c(c3ccc4cccc5ccc1c3c45)CCCC2,1,1
1,1,O=c1c2ccccc2c(=O)c2c1ccc1c2[nH]c2c3c(=O)c4cccc...,0,1
2,2,[N-]=[N+]=CC(=O)NCC(=O)NN,1,1
3,3,[N-]=[N+]=C1C=NC(=O)NC1=O,1,1
4,4,CCCCN(CC(O)C1=CC(=[N+]=[N-])C(=O)C=C1)N=O,1,1
...,...,...,...,...
7934,7934,O=c1[nH]c2cc(Cl)c(Cl)c([N+](=O)[O-])c2[nH]c1=O,1,3
7935,7935,C[S+](CC[C@H](N)C(=O)[O-])C[C@H]1O[C@@H](n2cnc...,1,3
7936,7936,CC(Cc1ccccc1)n1cc([N-]C(=O)Nc2ccccc2)[o+]n1,1,3
7937,7937,CCc1c(C)[n+]([NH-])c(-c2ccc(OC)c(OC)c2)c2cc(OC...,1,3


In [4]:
warnings.filterwarnings("ignore")
RDLogger.DisableLog("rdApp.*")


def get_decsriptors_df(smiles_list):
    descriptors_list = []

    for smiles in tqdm(smiles_list):
        descriptors_list.append(
            Descriptors.CalcMolDescriptors(Chem.MolFromSmiles(smiles), 0)
        )
    return pd.DataFrame(descriptors_list).fillna(0)

In [6]:
X = get_decsriptors_df(df["Drug"])
y = df["Y"]

100%|██████████| 7939/7939 [00:45<00:00, 176.40it/s]


In [12]:
test_df = pd.read_csv("data/test_data.csv")
test_df

,Unnamed: 0,Drug_ID,Drug,property
0,0,0,Cc1cc(=O)oc2ccc3c(c12)C1(C)OOC1(C)O3,1
1,1,1,CN(CC(O)CO)N=O,1
2,2,2,O=[N+]([O-])c1ccc(OC2CO2)cc1,1
3,3,3,Cn1cnc2cc([N+](=O)[O-])ccc21,1
4,4,4,CC[C@H](CO)NCCN[C@H](CC)CO,1
...,...,...,...,...
1216,1216,1216,CCN(CC)C(=O)COc1cc2c(O)c3c(O)c(C)c4c(c13)C(=O)...,3
1217,1217,1217,O=C(Cc1ccc(Cl)c(Cl)c1)N1CCc2sccc2[C@H]1CN1CCCC1,3
1218,1218,1218,CN1CC[C@]23c4c5ccc(O)c4O[C@H]2C(=O)CC[C@H]3[C@...,3
1219,1219,1219,Cn1nc(-c2cccc(F)c2)n(C)c1=S,3


In [7]:
X_test = get_decsriptors_df(test_df["Drug"])

100%|██████████| 1221/1221 [00:19<00:00, 61.31it/s]


In [9]:


cb_class = CatBoostClassifier(silent=True)
cb_class.fit(X, y)

In [15]:
sample = pd.read_csv("data/sample.csv")

,id,Y
0,0,1
1,1,0
2,2,0
3,3,0
4,4,1
...,...,...
1216,1216,0
1217,1217,0
1218,1218,0
1219,1219,0


In [24]:
sample["Y"] = cb_class.predict_proba(X_test)[:, 1]

In [26]:
sample.to_csv("sub_2.csv", index=False)